# ML Model Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [6]:
import influxdb_client_3 as InfluxDBClient3
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

Next we provide our connection credentials to the InfluxDB 3.0 instance. 
`token` - scoped to the bucket you wish to query from.
`host` - the host of your cloud dedicated instance.
`client` - host wrapped in the flightclient class. 

In [7]:
token = ''
host = 'eu-central-1-1.aws.cloud2.influxdata.com'    
org = 'Jay-IOx'
db = 'factory'

client = InfluxDBClient3.InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    


In [8]:
query = "SELECT vibration FROM machine_data WHERE \"machineID\" = 'machine2' AND time >= now() - 50m"
table = client.query(query=query, language="influxql")
print(table)

pyarrow.Table
iox::measurement: string not null
time: timestamp[ns] not null
vibration: double
----
iox::measurement: [["machine_data","machine_data","machine_data","machine_data","machine_data",...,"machine_data","machine_data","machine_data","machine_data","machine_data"]]
time: [[2023-06-28 09:54:35.296288245,2023-06-28 09:54:35.614346443,2023-06-28 09:54:36.614285038,2023-06-28 09:54:37.614883179,2023-06-28 09:54:38.615056141,...,2023-06-28 09:58:16.858854302,2023-06-28 09:58:17.859096470,2023-06-28 09:58:18.859357894,2023-06-28 09:58:19.859761125,2023-06-28 09:58:20.860034308]]
vibration: [[89,60,67,70,76,...,82,86,86,82,85]]


The `query_tools` function provides a wrapper to execution of a query agaisnt InfluxDB usinf flight. This function requires 3 paramters.
`database`- this specifies the database you wish to direct your query at.
`query` - This accepts a string literal of the query you wish to perform (SQL or InfluxQL).
`query_type` - Whether this is a SQL or InfluxQL based query.

Return all columns from the table flight for the past 3 days.

In [9]:
df = table.to_pandas()
display(df)

,iox::measurement,time,vibration
0,machine_data,2023-06-28 09:54:35.296288245,89.0
1,machine_data,2023-06-28 09:54:35.614346443,60.0
2,machine_data,2023-06-28 09:54:36.614285038,67.0
3,machine_data,2023-06-28 09:54:37.614883179,70.0
4,machine_data,2023-06-28 09:54:38.615056141,76.0
...,...,...,...
222,machine_data,2023-06-28 09:58:16.858854302,82.0
223,machine_data,2023-06-28 09:58:17.859096470,86.0
224,machine_data,2023-06-28 09:58:18.859357894,86.0
225,machine_data,2023-06-28 09:58:19.859761125,82.0


In [10]:
# Save the table to a Parquet file
pq.write_table(table, 'out.parquet' , version='2.6')

In [11]:
df.to_csv(index=False, 
path_or_buf='./out.csv')

In [12]:
result = df.to_json(orient="records", path_or_buf='./out.json', date_unit ='ns')


In [13]:
from pyarrow import orc
orc.write_table(table, 'out.orc')

In [14]:
from pyarrow import feather
feather.write_feather(table, 'out.feather')